In [1]:
# hyperparams
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import codecs
import regex
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

# data
source_train = 'drive/MyDrive/Corpus/corpus_1w/english1w.txt'
target_train = 'drive/MyDrive/Corpus/corpus_1w/chinese_out.txt'
source_test = 'drive/MyDrive/Corpus/corpus_1w/test_english.txt'
target_test = 'drive/MyDrive/Corpus/corpus_1w/test_chineseout.txt'

# training
batch_size = 32 # alias = N
lr = 0.0001 # learning rate.
logdir = 'drive/Shareddrives/ruochen.katherina@gmail.com/logdir_en_zh' # log directory

# model
maxlen = 30 # Maximum number of words in a sentence. alias = T.
min_cnt = 20 # words whose occurred less than min_cnt are encoded as <UNK>.
hidden_units = 512 # alias = C
num_blocks = 6 # number of encoder/decoder blocks
num_epochs = 20 # traversal time
num_heads = 8
dropout_rate = 0.1
sinusoid = False # select embedding method

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def load_cn_vocab():
    vocab = [line.split()[0] for line in codecs.open('drive/MyDrive/Corpus/corpus_1w/cn.vocab.tsv', 'r', 'utf-8').read().splitlines() if int(line.split()[1])>=min_cnt]
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for idx, word in enumerate(vocab)}
    return word2idx, idx2word

def load_en_vocab():
    vocab = [line.split()[0] for line in codecs.open('drive/MyDrive/Corpus/corpus_1w/en.vocab.tsv', 'r', 'utf-8').read().splitlines() if int(line.split()[1])>=min_cnt]
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for idx, word in enumerate(vocab)}
    return word2idx, idx2word

def create_data(source_sents, target_sents): 
    en2idx, idx2en = load_en_vocab()
    cn2idx, idx2cn = load_cn_vocab()
    
    # Index
    x_list, y_list, Sources, Targets = [], [], [], []
    for source_sent, target_sent in zip(source_sents, target_sents):
        x = [en2idx.get(word, 1) for word in (source_sent + u" </S>").split()] # 1: OOV, </S>: End of Text
        y = [cn2idx.get(word, 1) for word in (target_sent + u" </S>").split()] 
        if max(len(x), len(y)) <=maxlen:
            x_list.append(np.array(x))
            y_list.append(np.array(y))
            Sources.append(source_sent)
            Targets.append(target_sent)
    
    # Pad      
    X = np.zeros([len(x_list), maxlen], np.int32)
    Y = np.zeros([len(y_list), maxlen], np.int32)
    for i, (x, y) in enumerate(zip(x_list, y_list)):
        X[i] = np.lib.pad(x, [0, maxlen-len(x)], 'constant', constant_values=(0, 0))
        Y[i] = np.lib.pad(y, [0, maxlen-len(y)], 'constant', constant_values=(0, 0))
    
    return X, Y, Sources, Targets

def load_train_data():
    cn_sents = [regex.sub("<[^>]+>", "", line) for line in codecs.open(source_train, 'r', 'utf-8').read().split("\n") if line and line[0] != "<"]
    en_sents = [regex.sub("<[^>]+>", "", line) for line in codecs.open(target_train, 'r', 'utf-8').read().split("\n") if line and line[0] != "<"]
    
    X, Y, Sources, Targets = create_data(cn_sents, en_sents)
    return X, Y
    
def load_test_data():
    def _refine(line):
        line = regex.sub("<[^>]+>", "", line)
        line = regex.sub("<[^>]+>", "", line)
        return line.strip()
    
    cn_sents = [_refine(line) for line in codecs.open(source_test, 'r', 'utf-8').read().split("\n") if line ]
    en_sents = [_refine(line) for line in codecs.open(target_test, 'r', 'utf-8').read().split("\n") if line ]
        
    X, Y, Sources, Targets = create_data(cn_sents, en_sents)
    return X, Sources, Targets # (1064, 150) 

def load_test_data1():
    def _refine(line):
        line = regex.sub("<[^>]+>", "", line)
        line = regex.sub("<[^>]+>", "", line)
        return line.strip()
    
    cn_sents = [_refine(line) for line in codecs.open(source_test1, 'r', 'utf-8').read().split("\n") if line ]
    en_sents = [_refine(line) for line in codecs.open(target_test1, 'r', 'utf-8').read().split("\n") if line ]
        
    X, Y, Sources, Targets = create_data(cn_sents, en_sents)
    return X, Sources, Targets # (1064, 150) 

def get_batch_data():
    X, Y = load_train_data()
    
    # calculate batch size
    num_batch = len(X) // batch_size
    
    X = tf.convert_to_tensor(X, tf.int32)
    Y = tf.convert_to_tensor(Y, tf.int32)
    
    input_queues = tf.compat.v1.train.slice_input_producer([X, Y])
            
    x, y = tf.compat.v1.train.shuffle_batch(input_queues,
                                num_threads=8,
                                batch_size=batch_size, 
                                capacity=batch_size*64,   
                                min_after_dequeue=batch_size*32, 
                                allow_smaller_final_batch=False)
    
    return x, y, num_batch # (N, T), (N, T), ()

In [3]:
# layer normalization
def normalize(inputs, epsilon = 1e-8, scope="ln", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]
    
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta= tf.Variable(tf.zeros(params_shape))
        gamma = tf.Variable(tf.ones(params_shape))
        normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
        outputs = gamma * normalized + beta
        
    return outputs

# initial embedding operation
def embedding(inputs, vocab_size, num_units, zero_pad=True, scale=True,scope="embedding", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        lookup_table = tf.get_variable('lookup_table',
                                       dtype=tf.float32,
                                       shape=[vocab_size, num_units],
                                       initializer=tf.keras.initializers.glorot_normal()) 
        if zero_pad:
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:, :]), 0)
        outputs = tf.nn.embedding_lookup(lookup_table, inputs) # complete word embedding, change into 3d tensor
        
        if scale:
            outputs = outputs * (num_units ** 0.5) # scale
            
    return outputs
    
# positional encoding
def positional_encoding(inputs, num_units,zero_pad=True, scale=True, scope="positional_encoding", reuse=None):

    N, T = inputs.get_shape().as_list()
    with tf.variable_scope(scope, reuse=reuse):
        
        #position indices
        position_ind = tf.tile(tf.expand_dims(tf.range(T), 0), [N, 1])

        position_enc = np.array([
            [pos / np.power(10000, 2.*i/num_units) for i in range(num_units)]
            for pos in range(T)])

        position_enc[:, 0::2] = np.sin(position_enc[:, 0::2])  
        position_enc[:, 1::2] = np.cos(position_enc[:, 1::2])  

        lookup_table = tf.convert_to_tensor(position_enc)

        if zero_pad:
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:, :]), 0)
        outputs = tf.nn.embedding_lookup(lookup_table, position_ind)

        if scale:
            outputs = outputs * num_units**0.5

        return outputs


# attention mechanism
def multihead_attention(queries, keys, num_units=None, num_heads=8, dropout_rate=0, is_training=True, causality=False, scope="multihead_attention", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list[-1]
        
        # Linear projections
        Q = tf.layers.dense(queries, num_units, activation=tf.nn.relu) # (N, T_q, C)
        K = tf.layers.dense(keys, num_units, activation=tf.nn.relu) # (N, T_k, C)
        V = tf.layers.dense(keys, num_units, activation=tf.nn.relu) # (N, T_k, C)
        
        # Split and concat
        Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 

        # Multiplication
        outputs = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
        
        # Scale
        outputs = outputs / (K_.get_shape().as_list()[-1] ** 0.5)
        
        # Key Masking
        key_masks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1))) # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1]) # (h*N, T_k)
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, tf.shape(queries)[1], 1]) # (h*N, T_q, T_k)
        
        paddings = tf.ones_like(outputs)*(-2**32+1)
        outputs = tf.where(tf.equal(key_masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Causality = Future blinding
        if causality:
            diag_vals = tf.ones_like(outputs[0, :, :]) # (T_q, T_k)
            tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1]) # (h*N, T_q, T_k)
   
            paddings = tf.ones_like(masks)*(-2**32+1)
            outputs = tf.where(tf.equal(masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Activation
        outputs = tf.nn.softmax(outputs) # (h*N, T_q, T_k)
         
        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(queries, axis=-1))) # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1]) # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(keys)[1]]) # (h*N, T_q, T_k)
        outputs *= query_masks # broadcasting. (h*N, T_q, T_k)
          
        # Dropouts avoid overfitting
        outputs = tf.layers.dropout(outputs, rate=dropout_rate, training=tf.convert_to_tensor(is_training))
               
        # Weighted sum
        outputs = tf.matmul(outputs, V_) # ( h*N, T_q, C/h)
        
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2 ) # (N, T_q, C)
              
        # Residual connection
        outputs += queries
              
        # Normalize
        outputs = normalize(outputs) # (N, T_q, C)
 
    return outputs


def feedforward(inputs, num_units=[2048, 512],scope="multihead_attention", reuse=None):

    with tf.variable_scope(scope, reuse=reuse):
        # Inner layer
        params = {"inputs": inputs, "filters": num_units[0], "kernel_size": 1,
                  "activation": tf.nn.relu, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Readout layer
        params = {"inputs": outputs, "filters": num_units[1], "kernel_size": 1,
                  "activation": None, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Residual connection
        outputs += inputs
        
        # Normalize
        outputs = normalize(outputs)
    
    return outputs

# smoothing
def label_smoothing(inputs, epsilon=0.1):
    ''' https://arxiv.org/abs/1512.00567.'''

    K = inputs.get_shape().as_list()[-1] # number of channels
    return ((1-epsilon) * inputs) + (epsilon / K)

In [5]:
class Graph():
    def __init__(self, is_training=True):
        self.graph = tf.Graph()
        with self.graph.as_default():
            if is_training:
                self.x, self.y, self.num_batch = get_batch_data() # (N, T)
            else: # inference
                self.x = tf.placeholder(tf.int32, shape=(None, maxlen))
                self.y = tf.placeholder(tf.int32, shape=(None, maxlen))

            # decoder input
            self.decoder_inputs = tf.concat((tf.ones_like(self.y[:, :1])*2, self.y[:, :-1]), -1) # 2:<S>

            # load   
            cn2idx, idx2cn = load_cn_vocab()
            en2idx, idx2en = load_en_vocab()
            
            # Encoder
            with tf.variable_scope("encoder"):
                ## embedding
                self.enc = embedding(self.x, 
                                      vocab_size=len(en2idx), 
                                      num_units=hidden_units, 
                                      scale=True,
                                      scope="enc_embed")
                
                ## positional embedding
                if sinusoid:
                    self.enc += positional_encoding(self.x,
                                      num_units=hidden_units, 
                                      zero_pad=False, 
                                      scale=False,
                                      scope="enc_pe")
                else:
                    self.enc += embedding(tf.tile(tf.expand_dims(tf.range(tf.shape(self.x)[1]), 0), [tf.shape(self.x)[0], 1]),
                                      vocab_size=maxlen, 
                                      num_units=hidden_units, 
                                      zero_pad=False, 
                                      scale=False,
                                      scope="enc_pe")
                    
                 
                ## dropout layer
                self.enc = tf.layers.dropout(self.enc, 
                                            rate=dropout_rate, 
                                            training=tf.convert_to_tensor(is_training))
                
                ## number of encoders: num_blocks
                for i in range(num_blocks):
                    with tf.variable_scope("num_blocks_{}".format(i)):
                        ### Multihead Attention
                        self.enc = multihead_attention(queries=self.enc, 
                                                        keys=self.enc, 
                                                        num_units=hidden_units, 
                                                        num_heads=num_heads, 
                                                        dropout_rate=dropout_rate,
                                                        is_training=is_training,
                                                        causality=False)
                        
                        ### Feed Forward
                        self.enc = feedforward(self.enc, num_units=[4*hidden_units, hidden_units])
            
            # decoder
            with tf.variable_scope("decoder"):
                self.dec = embedding(self.decoder_inputs, 
                                      vocab_size=len(cn2idx), 
                                      num_units=hidden_units,
                                      scale=True, 
                                      scope="dec_embed")
                
                if sinusoid:
                    self.dec += positional_encoding(self.decoder_inputs,
                                      vocab_size=maxlen, 
                                      num_units=hidden_units, 
                                      zero_pad=False, 
                                      scale=False,
                                      scope="dec_pe")
                else:
                    self.dec += embedding(tf.tile(tf.expand_dims(tf.range(tf.shape(self.decoder_inputs)[1]), 0), [tf.shape(self.decoder_inputs)[0], 1]),
                                      vocab_size=maxlen, 
                                      num_units=hidden_units, 
                                      zero_pad=False, 
                                      scale=False,
                                      scope="dec_pe")
                
                self.dec = tf.layers.dropout(self.dec, 
                                            rate=dropout_rate, 
                                            training=tf.convert_to_tensor(is_training))
                
                for i in range(num_blocks):
                    with tf.variable_scope("num_blocks_{}".format(i)):
                        ## Multihead Attention ( self-attention)
                        self.dec = multihead_attention(queries=self.dec, 
                                                        keys=self.dec, 
                                                        num_units=hidden_units, 
                                                        num_heads=num_heads, 
                                                        dropout_rate=dropout_rate,
                                                        is_training=is_training,
                                                        causality=True, 
                                                        scope="self_attention")
                        
                        ## Multihead Attention (vanilla attention)
                        self.dec = multihead_attention(queries=self.dec, 
                                                        keys=self.enc, 
                                                        num_units=hidden_units, 
                                                        num_heads=num_heads,
                                                        dropout_rate=dropout_rate,
                                                        is_training=is_training, 
                                                        causality=False,
                                                        scope="vanilla_attention")
                        
                        ## Feed Forward
                        self.dec = feedforward(self.dec, num_units=[4*hidden_units, hidden_units])
                
            # Final linear projection
            self.logits = tf.layers.dense(self.dec, len(cn2idx)) #shape: [N,T,len(en2idx)]
            self.preds = tf.to_int32(tf.arg_max(self.logits, dimension=-1))
            self.istarget = tf.to_float(tf.not_equal(self.y, 0))
            self.acc = tf.reduce_sum(tf.to_float(tf.equal(self.preds, self.y))*self.istarget)/ (tf.reduce_sum(self.istarget))
            tf.summary.scalar('acc', self.acc)
            
            if is_training:  
                # smoothing
                self.y_smoothed = label_smoothing(tf.one_hot(self.y, depth=len(cn2idx)))
                self.loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y_smoothed)
                self.mean_loss = tf.reduce_sum(self.loss*self.istarget) / (tf.reduce_sum(self.istarget))
               
                # graph
                self.global_step = tf.Variable(0, name='global_step', trainable=False)
                self.optimizer = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.9, beta2=0.98, epsilon=1e-8)
                self.train_op = self.optimizer.minimize(self.mean_loss, global_step=self.global_step)
                   
                # summary
                tf.summary.scalar('mean_loss', self.mean_loss)
                self.merged = tf.summary.merge_all()

# if __name__ == '__main__':                    
#     cn2idx, idx2cn = load_cn_vocab()
#     en2idx, idx2en = load_en_vocab()
    
#     g = Graph("train"); print("load successfully")
    
#     sv = tf.train.Supervisor(graph=g.graph, 
#                              logdir=logdir,
#                              save_model_secs=0)
#     with sv.managed_session() as sess:
#         for epoch in range(1, num_epochs+1): 
#             print('The%d' % (epoch)+'period')
#             if sv.should_stop(): break
#             for step in tqdm(range(g.num_batch), total=g.num_batch, ncols=70, leave=False, unit='b'):
#                 sess.run(g.train_op)
                
#             gs = sess.run(g.global_step)   
#             sv.saver.save(sess, logdir + '/model_epoch_%02d_gs_%d' % (epoch, gs))
    
#     print("Done")    

In [7]:
import os
def eval(): 
    # load graph
    g = Graph(is_training=False)
    print("Graph loaded")
    
    # load data
    X, Sources, Targets = load_test_data()
    cn2idx, idx2cn = load_cn_vocab()
    en2idx, idx2en = load_en_vocab()
     
     
    # Start session 
    with g.graph.as_default():    
        sv = tf.train.Supervisor()
        with sv.managed_session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
            ## restore parameters
            sv.saver.restore(sess, tf.train.latest_checkpoint(logdir))
            print("Restored!")
              
            ## Get model name
            mname = open(logdir + '/checkpoint', 'r').read().split('"')[1] # model name
             
            ## Inference
            if not os.path.exists('drive/Shareddrives/ruochen.katherina@gmail.com/results_en_zh'): os.mkdir('drive/Shareddrives/ruochen.katherina@gmail.com/results_en_zh')
            with codecs.open("drive/Shareddrives/ruochen.katherina@gmail.com/results_en_zh/" + mname, "w", "utf-8") as fout:
                list_of_refs, hypotheses = [], []
                for i in range(len(X) // batch_size):
                     
                    ### Get mini-batches
                    x = X[i*batch_size: (i+1)*batch_size]
                    sources = Sources[i*batch_size: (i+1)*batch_size]
                    targets = Targets[i*batch_size: (i+1)*batch_size]
                     
                    ### Autoregressive inference
                    preds = np.zeros((batch_size, maxlen), np.int32)
                    for j in range(maxlen):
                        _preds = sess.run(g.preds, {g.x: x, g.y: preds})
                        preds[:, j] = _preds[:, j]
                     
                    ### Write to file
                    for source, target, pred in zip(sources, targets, preds): # sentence-wise
                        got = " ".join(idx2cn[idx] for idx in pred).split("</S>")[0].strip()
                        fout.write("- source: " + source +"\n")
                        fout.write("- expected: " + target + "\n")
                        fout.write("- got: " + got + "\n\n")
                        print("- source: " + source +"\n")
                        print("- expected: " + target + "\n")
                        print("- got: " + got + "\n\n")
                        fout.flush()
                          
                        #bleu score
                        ref = target.split()
                        hypothesis = got.split()
                        if len(ref) > 3 and len(hypothesis) > 3:
                            list_of_refs.append([ref])
                            hypotheses.append(hypothesis)
              
                ## Calculate bleu score
                print(list_of_refs)
                score = corpus_bleu(list_of_refs, hypotheses)
                fout.write("Bleu Score = " + str(100*score))
                print("Bleu Score = " + str(100*score))
                                          
if __name__ == '__main__':
    eval()
    print("Done")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:268: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  warnings.warn('`tf.layers.dropout` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:202: UserWarning: `tf.layers.conv1d` is deprecated and will be removed 

Graph loaded
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Restoring parameters from drive/Shareddrives/ruochen.katherina@gmail.com/logdir_en_zh/model_epoch_20_gs_2640
Restored!
- source: he who seeks " taiwan independence " will come to no good end .

- expected: 谁 搞 " 台独 " , 谁 就 没有 好下场 .

- got: 谁 搞 " 台独 " , 谁 就 没有 <UNK> .


- source: qian qichen hoped the people of all sectors in hong kong will make positive contribution to strengthening cross - strait cooperation and exchanges in various fields .

- expected: 钱其琛 希望 香港 各界人士 在 加强 两岸 各 领域 合作 与 交流 方面 做出 积极 的 贡献 .

- got: 钱其琛 希望 香港 <UNK> 在 加强 两岸 各 领域 合作 与 交流 方面 做出 积极 的 贡献 .


- source: the dprk guests arrived china for a visit at the invitation of the cyl central committee .

- expected: 朝鲜 客人 是 应 共青团中央 的 邀请 来华访问 的 .

- got: 朝鲜 <UNK> 是 应 <UNK> 的 邀请 , 开始 <UNK> 的 .


- source: the two sides exchanged vi